In [ ]:
class bitkubAPI():
    def __init__(self,host,api_key,api_secret):
        self.API_HOST = host
        self.API_KEY = api_key
        self.API_SECRET = api_secret
        self.header = {
            'Accept': 'application/json',
            'Content-Type': 'application/json',
            'X-BTK-APIKEY': self.API_KEY,
            }
        
    def timestamp(self,tm):
        date = datetime.datetime.strptime(tm, "%Y-%m-%d")
        ts = time.mktime(date.timetuple())
        return ts
        
    def _json_encode(self,data):
        return json.dumps(data, separators=(',', ':'), sort_keys=True)

    def _sign(self,data):
        j = self._json_encode(data)
        h = hmac.new(API_SECRET, msg=j.encode(), digestmod=hashlib.sha256)
        return h.hexdigest()
    
    def _get(self,url,**kwargs): 
        try:
            res = requests.get(self.API_HOST+ url,**kwargs)
            res = json.loads(res.text)
            res = res
            return res
        except:
            return res
    
    def _post(self,url,data): 
        try:
            signature = self._sign(data)
            data['sig'] = signature
            res = requests.post(self.API_HOST + url, headers= self.header, data=self._json_encode(data))
            return res
        except:
            return res
        
    def api_status(self):  
        return self._get('/api/status')
        
    def serverTime(self):
        res = requests.get(self.API_HOST + '/api/servertime')
        return json.loads(res.text)
    
    def bids(self,sym):
        return self._get('/api/market/bids?sym='+sym+'&lmt=1')

    def asks(self,sym):
        return self._get('/api/market/asks?sym='+sym+'&lmt=1')
    
    def balance(self):
        data = {
            'ts': self.getServerTime(),
        }
        return self._post('/api/market/balances',data)
    
    def opened_orders(self,sym):
        data = {
            'sym': sym,
        }
        return self._post('/api/market/balances',data)
    
    def info_orders(self,sym,order_id,side,order_hash):
        data = {
            'sym': sym,
            'id':order_id,
            'sd':side,
            'hash':order_hash,
        }
        return self._post('/api/market/balances',data)
    
    def cancel_order(self,sym,order_id,side,order_hash):
        data = {
            'sym': sym,
            'id':order_id,
            'sd':side,
            'hash':order_hash,
        }
        return self._post('/api/market/cancel-order',data)
    
    def his_price(self,sym,frm):
        params = 'sym='+sym
        params = params + '&int=86400'
        params = params + f'&frm={int(self.timestamp(frm))}'
        params = '/api/market/tradingview?'+params
        return self._get(params)
    
    def place_order(self,sym,side,amt,rat,typ):
        data = {
            'sym': sym,
            'amt':amt,
            'rat':rat,
            'typ':typ,
        }
        if(side =='buy'):
            return self._post('/api/market/place-ask/test',data)
        elif(side =='sell'):
            return self._post('/api/market/place-bid/test',data)